In [30]:
import numpy as np
import cv2 as cv
import glob

np.set_printoptions(suppress=True) 

In [31]:
#Extrinsic Parametrers
#Matrices de rotacion para la camara 
Angle_x = np.deg2rad(180)
Angle_y = np.deg2rad(0)
Angle_z = np.deg2rad(180)
Rx = np.array([[1,0,0],
               [0,np.cos(Angle_x),-np.sin(Angle_x)],
               [0,np.sin(Angle_x),np.cos(Angle_x)],
              ])
Ry = np.array([
                [np.cos(Angle_y),0,np.sin(Angle_y)],
               [0,1,0],
               [-np.sin(Angle_y),0,np.cos(Angle_y)],
              ])
Rz = np.array([
                [np.cos(Angle_z),-np.sin(Angle_z),0],
                [np.sin(Angle_z),np.cos(Angle_z),0],
                [0,0,1]
            ])

In [32]:
print(Rx)
print(Ry)
print(Rz)


[[ 1.  0.  0.]
 [ 0. -1. -0.]
 [ 0.  0. -1.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
[[-1. -0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]


In [33]:
Rotation_matrix = Rz.dot(Ry).dot(Rx)# calculando matriz de rotacion

In [34]:
print(Rotation_matrix)

[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]


In [35]:
# Vector de posiscion del origen de la camara 
X_o=np.array([[100],[0],[2000]])
# vector de posicion del objeto
X_p= np.array([[650],[200],[0]])

In [36]:
print("X_o")
print(X_o)
print("X_p")
print(X_p)

X_o
[[ 100]
 [   0]
 [2000]]
X_p
[[650]
 [200]
 [  0]]


In [37]:
#Acomodandolo las matrices en la frorma homogenea 
#Matriz de rotacion Homogenea
H_Rotation_Matrix=np.append(Rotation_matrix,np.zeros((1,3)),axis=0)#agregando vector de 0 en la base
H_Rotation_Matrix=np.append(H_Rotation_Matrix,np.array([[0],[0],[0],[1]]), axis = 1)#agregando un vector al lateral

In [38]:
print(H_Rotation_Matrix)

[[-1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]


In [39]:
#Matriz de proyeccion del centro de camara
Auxiliar_X_o = np.array([np.append(-X_o,np.array([1]))])
H_X_o = np.append(np.eye(3), np.zeros((1,3)), axis=0)
H_X_o = np.append(H_X_o, Auxiliar_X_o.T, axis=1)

In [40]:
print(H_X_o)

[[    1.     0.     0.  -100.]
 [    0.     1.     0.     0.]
 [    0.     0.     1. -2000.]
 [    0.     0.     0.     1.]]


In [41]:
#Vector  X_p en forma homogenea
H_X_p = np.array([np.append(X_p,1)])
print(H_X_p)

[[650 200   0   1]]


In [42]:
#Matriz que expresa el vector de posicion del objeto en las coordenadas de la camra
print(H_Rotation_Matrix.dot(H_X_o))
K_X_p = H_Rotation_Matrix.dot(H_X_o).dot(H_X_p.T)
print(K_X_p)

[[  -1.    0.    0.  100.]
 [   0.    1.    0.   -0.]
 [   0.    0.   -1. 2000.]
 [   0.    0.    0.    1.]]
[[-550.]
 [ 200.]
 [2000.]
 [   1.]]


In [43]:
#Intrinsic parametrers
# funciones para segmentar color y buscar el centroide
def seg_color(src,bajos, altos):
    mask = cv.inRange(src, bajos, altos)
    return mask
        
def buscar_centroide(mask):

    moments = cv.moments(mask)
    area=moments['m00']
    
    x = int(moments['m10']/moments['m00'])
    y = int(moments['m01']/moments['m00'])
    
    # put text and highlight the center

    #cv.circle(mask, (x, y), 5, (0, 0, 0), -1)

    #cv.putText(mask, "centroid", (x - 25, y - 25),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv.rectangle(mask,(x,y),(x+2, y+2),(0,0,0),2)#poner un punto en el centroide
    
    
    return mask,x,y
   

In [44]:
#rangos de rojo que queremos segmentar 
rojo_bajos = np.array([0,0,80], dtype=np.uint8)
rojo_altos = np.array([240, 122, 255], dtype=np.uint8)


In [45]:
#leyendo imagen de robo dk y tomando la posicion del centroide,
#estas coordenadas van a ser utiles para calcular la constante de la camara

src=cv.imread('Frame 3-Snapshot_1.png')

src_np= np.array((src))
print(src_np.shape)
maskRojo=seg_color(src, rojo_bajos, rojo_altos) 
print(maskRojo.shape)
maskRojo,xRojo,yRojo=buscar_centroide(maskRojo) 


print(xRojo,yRojo)
cv.imshow('camara',src)
cv.imshow('Rojo', maskRojo)
cv.waitKey(0)
cv.destroyAllWindows()

(480, 640, 3)
(480, 640)
139 305


In [46]:
#Moviendo el 0,0 de la imagen al centro
y_center= int(src.shape[0]/2)
x_center= int(src.shape[1]/2)
print(x_center,y_center)
img = np.zeros((480,640,3), np.uint8)
cv.line(src,(x_center,0),(x_center,480),(255,0,0),1)
cv.line(src,(0,y_center),(640,y_center),(255,0,0),1)
cv.imshow('axis',src)
cv.waitKey(0)
cv.destroyAllWindows()

320 240


In [47]:
#ubicando el centroide del objeto con el nuevo centro de la imagen
#conocido tambn como cambio de base
N_xRojo=xRojo-x_center#estos son los valores q se deben usar para la matriz de transformacion homogenia para pasar de los valores 2d a 3d
N_yRojo=yRojo-y_center
print(N_xRojo,N_yRojo)
cv.rectangle(src,(xRojo,yRojo),(xRojo+2, yRojo+2),(255,0,0),2)#poner un punto en el centroide
cv.imshow('Centroide',src)
cv.waitKey(0)
cv.destroyAllWindows()

-181 65


In [48]:
#Encontrando la constante de la camara
#debido a que [x*w,y*w,w].T=[[c,0,0,0],[0,c,0,0],[0,0,1,0]].*[X,Y,Z,1].T 
#sabemos que Z==w y teniendo el punto en la imajen (N_xRojo,N_yRojo) y el vector K_X_p
#podemos encontrar el valor de c que es la constante de la camara
np.set_printoptions(suppress=False) 
c=float(N_xRojo*K_X_p[2]/K_X_p[0])
print(K_X_p)
print(c)

[[-5.5e+02]
 [ 2.0e+02]
 [ 2.0e+03]
 [ 1.0e+00]]
658.1818181818181


In [49]:
#comprobando
C_x_p=(np.array([[c,0,0,0],[0,c,0,0],[0,0,1,0]]).dot(K_X_p))
C_X_p=(np.array([[c,0,0,0],[0,c,0,0],[0,0,1,0]]))
print(C_X_p,'\n')
#dividiendo por el valor Z

print(C_x_p)
print(C_x_p/K_X_p[2])

[[658.18181818   0.           0.           0.        ]
 [  0.         658.18181818   0.           0.        ]
 [  0.           0.           1.           0.        ]] 

[[-362000.        ]
 [ 131636.36363636]
 [   2000.        ]]
[[-181.        ]
 [  65.81818182]
 [   1.        ]]


In [50]:
error= ((65-65.81818182)/65)*100
error2= ((181-178.75)/181)*100
print(error2)
print(error)

1.2430939226519337
-1.2587412615384717


In [51]:
#corrigiendo error
#para corregirlo debemos realizar la traslacion del centro de la imagen pero por medio de coordenadas homogeneas
#ademas de agregar un parametro de diferencia de escala m que tiene la camara entre x y y

In [53]:
S_H_c=np.array([[1,0,x_center],[0,1,y_center],[0,0,1]]).dot(C_x_p)

S_H_c1_matriz=np.array([[1,0,x_center],[0,1,y_center],[0,0,1]])
print(S_H_c1_matriz)
print(S_H_c)
print(S_H_c/K_X_p[2])#estos son los valores del centrodie del objeto en la imagen con las coordenadas originales de la imagen


[[  1   0 320]
 [  0   1 240]
 [  0   0   1]]
[[278000.        ]
 [611636.36363636]
 [  2000.        ]]
[[139.        ]
 [305.81818182]
 [  1.        ]]


In [54]:
#se agrega un parametro m en la matriz anterior para ajustar la diferencia de escarla entre x e y
#tambien se agrega un parametro s que se conoce como "sheer" pero como estamos usando una camara digital este valor se asume como 0
#la matriz queda de la siguiente manera
#S_H_c_modificada=np.array([[1,s,x_center],[0,1+m,y_center],[0,0,1]])
#calculando m
s=0
m=float((((305*C_x_p[2])-(y_center*C_x_p[2]))/C_x_p[1])-1)
U=((0+(1+m)*C_x_p[1]+y_center*C_x_p[2]))/C_x_p[2]

print(m)
print(U)
print(C_x_p[2])
print(y_center)

-0.012430939226518278
[305.]
[2000.]
240


In [55]:
S_H_c2_matriz=np.array([[1,s,x_center],[0,1+m,y_center],[0,0,1]])#agregando el parametro m 

print(S_H_c2_matriz)

[[  1.           0.         320.        ]
 [  0.           0.98756906 240.        ]
 [  0.           0.           1.        ]]


In [56]:
#Calculando matriz de calibracion, esta matriz es una transformacion afín para corregir errores
K=S_H_c2_matriz.dot(C_X_p)#.dot(K_X_p)
print(K)

[[658.18181818   0.         320.           0.        ]
 [  0.         650.         240.           0.        ]
 [  0.           0.           1.           0.        ]]


In [57]:
#comprobando
S_H_c_modificado=np.array([[1,s,x_center],[0,1+m,y_center],[0,0,1]]).dot(C_x_p)
print(np.array([[1,s,x_center],[0,1+m,y_center],[0,0,1]]))
print(S_H_c_modificado,'\n')
print(S_H_c_modificado/S_H_c_modificado[2])#listo el algoritmo DLT

[[  1.           0.         320.        ]
 [  0.           0.98756906 240.        ]
 [  0.           0.           1.        ]]
[[278000.]
 [610000.]
 [  2000.]] 

[[139.]
 [305.]
 [  1.]]


In [58]:
#Direct linear transform, justamos todas las matrices calculadas a travez de los parametros
##Extrinsecas
#Matriz de posicion de la proyeccion de la camara, H_X_o
#Matriz de rotacion de la proyeccion de la camara, H_Rotation_Matrix
##intrinsecas
#Matriz de proyeccion de perspectiva ideal, C_X_p
#Matriz de calibracion, S_H_c1_matriz-> S_H_c2_matriz*C_X_p-> K
np.set_printoptions(suppress=True) 
P=K.dot(H_Rotation_Matrix).dot(H_X_o)
print(P)
#comprobando
print(P.dot(H_X_p.T))

print(P.dot(H_X_p.T)/2000)

[[  -658.18181818      0.           -320.         705818.18181818]
 [     0.            650.           -240.         480000.        ]
 [     0.              0.             -1.           2000.        ]]
[[278000.]
 [610000.]
 [  2000.]]
[[139.]
 [305.]
 [  1.]]


In [80]:
KR=K.dot(H_Rotation_Matrix)
KR_inv = np.linalg.inv(KR[:3,:3])
#KR_inv = np.append(KR_inv, np.zeros((3,1)), axis = 1)
x= np.array([[139],[305],[1]])
print(KR)
print(KR_inv)
print(x)
p_inv= X_o+2000*KR_inv.dot(x)
print(p_inv)

[[-658.18181818    0.         -320.            0.        ]
 [   0.          650.         -240.            0.        ]
 [   0.            0.           -1.            0.        ]]
[[-0.00151934  0.          0.48618785]
 [ 0.          0.00153846 -0.36923077]
 [ 0.          0.         -1.        ]]
[[139]
 [305]
 [  1]]
[[650.]
 [200.]
 [  0.]]
